Adapted from the work of:

*   Steve Phelps: https://github.com/phelps-sg/python-bigdata
*   rpi.analyticsdojo.com https://colab.research.google.com/github/RPI-DATA/course-intro-ml-app/blob/master/content/notebooks/18-big-data/01-intro-mapreduce.ipynb?pli=1#scrollTo=CW_a5y0NcPIW




# Overview

1. The Map-Reduce programming model
2. Recap of functional programming in Python
3. Python's `map` and `reduce` functions
4. Writing parallel code using `map`

# Background
- map and reduce primitives present in Lisp and many other functional languages. They were introduced in the 1960s.

- The Map-Reduce programming paradigm was popularised by Dean and Ghemawat 2008 in their paper "MapReduce: simplified data processing on large clusters"

- The first popular open-source implementation was Apache Hadoop, first released in 2011.



# Implementing reduce
## Linear reduce

- The `reduce` function in functional programming is also termed `fold`. Data can be folded *left* or *right*.

- The following implements a *left* fold.

In [ ]:
def foldl(f, data):
    # Case 1: empty list → error, because there is nothing to fold
    if len(data) == 0:
        raise TypeError("foldl() of empty sequence with no initial value")

    # Case 2: only one element left → return it as the final result
    elif len(data) == 1:
        return data[0]

    # Case 3: at least two elements → apply folding
    else:
        # Split list into head (first element) and tail (rest of the list)
        head = data[0]
        tail = data[1:]

        # Debug: show what we are currently folding
        print("Folding", head, "with", tail)

        # Apply the function f to the first two elements (head and first of tail)
        partial_result = f(head, tail[0])

        # Debug: show result of this step
        print("Partial result is", partial_result)

        # Recursive call:
        # Create a new list with the partial result + the remaining tail
        # Continue folding until only one element remains
        return foldl(f, [partial_result] + tail[1:])



### Testing `fold` function: + operator

let's have a look at the behavior of the fold function.

- We execute a fold opertaion with a + operation:

In [ ]:
foldl(lambda x, y: x + y, [1, 2, 3, 4, 5])

Folding 1 with [2, 3, 4, 5]
Partial result is 3
Folding 3 with [3, 4, 5]
Partial result is 6
Folding 6 with [4, 5]
Partial result is 10
Folding 10 with [5]
Partial result is 15


15


### Testing `fold` function: - operator (Subtraction)

let's have a look at the behavior of the fold function.

- We execute a fold opertaion with a + operation:

In [ ]:
foldl(lambda x, y: x - y, [1, 2, 3, 4, 5])

Folding 1 with [2, 3, 4, 5]
Partial result is -1
Folding -1 with [3, 4, 5]
Partial result is -4
Folding -4 with [4, 5]
Partial result is -8
Folding -8 with [5]
Partial result is -13


-13

▶  **Exercise**: what happens, if instead a *`left`* fold, a *`right`* fold is executed?

*   write the `foldr` function





In [ ]:
def foldr(f, data):
    if len(data) == 0:
        raise TypeError("foldr() of empty sequence with no initial value")
    elif len(data) == 1:
        return data[0]
    else:
        head = data[-1]
        tail = data[:-1]
        print("Folding", head, "with", tail)
        folded_tail = foldr(f, tail)
        print("Partial result is", f(head, folded_tail))
        return f(head, folded_tail)


In [ ]:
foldr(lambda x, y: x - y, [1, 2, 3, 4, 5])

Folding 5 with [1, 2, 3, 4]
Folding 4 with [1, 2, 3]
Folding 3 with [1, 2]
Folding 2 with [1]
Partial result is 1
Partial result is 2
Partial result is 2
Partial result is 3


3

## parallel reduce

- The following implements a *parallel* reduce.
- This structure is closer to real MapReduce, where reducers handle partitions in parallel and then merge result

In [ ]:
def fold_tree(f, data):
    # Empty → error
    if len(data) == 0:
        raise TypeError("fold_tree() of empty sequence with no initial value")

    # Single element → return it
    if len(data) == 1:
        return data[0]

    # Pair up elements like a tree
    new_data = []
    for i in range(0, len(data), 2):
        if i + 1 < len(data):
            # Combine pair (data[i], data[i+1])
            new_data.append(f(data[i], data[i+1]))
            print(f"Folding {data[i]} with {data[i+1]} → {data[i], data[i+1]}")
        else:
            # Odd element, carry forward
            new_data.append(data[i])

    # Recurse until one element remains
    return fold_tree(f, new_data)

In [ ]:
fold_tree(lambda x, y: x + y, [1, 2, 3, 4, 5, 6])

Folding 1 with 2 → (1, 2)
Folding 3 with 4 → (3, 4)
Folding 5 with 6 → (5, 6)
Folding 3 with 7 → (3, 7)
Folding 10 with 11 → (10, 11)


21

## Executing Map-Reduce

- let's execute map-reduce with a colorlist example using the fold-function


In [ ]:
colorlist = ['Red', 'Blue', 'Green', 'Green', 'Green', 'Green', 'Green', 'Green',
 'Yellow', 'Black', 'White', 'Brown', 'Orange', 'Purple', 'Pink', 'Gray',
 'Cyan', 'Magenta', 'Silver', 'Gold', 'Indigo', 'Violet', 'Teal', 'Blue',
 'Blue', 'Blue', 'Blue', 'Blue', 'Blue', 'Blue', 'Maroon', 'Navy', 'Blue',
 'Aqua', 'Beige', 'Coral', 'Coral', 'Coral', 'Coral', 'Coral', 'Mint',
 'Olive', 'Lime', 'Turquoise', 'Burgundy', 'Crimson', 'Amber', 'Amber',
 'Amber']
colorlist = ['Red', 'Blue', 'Green', 'Green', 'Green', 'Red']

In [ ]:
from functools import reduce

### MAP-PHASE

*   Each input (e.g., a word) is converted into a (key, value)
pair.

In [ ]:
# -----------------------------
# Map step
# -----------------------------
def map_step(data):
    return list(map(lambda word: (word, 1), data))

In [ ]:
reduce()

In [ ]:
mapped = map_step(colorlist)
print("MAP →", mapped)

MAP → [('Red', 1), ('Blue', 1), ('Green', 1), ('Green', 1), ('Green', 1), ('Green', 1), ('Green', 1), ('Green', 1), ('Yellow', 1), ('Black', 1), ('White', 1), ('Brown', 1), ('Orange', 1), ('Purple', 1), ('Pink', 1), ('Gray', 1)]


### GROUP (SHUFFLE)-PHASE
- All values are grouped by key

In [ ]:
def groupByKey(data):
    result = {}
    for key, value in data:
        if key in result:
            result[key].append(value)
        else:
            result[key] = [value]
    return result

grouped = groupByKey(mapped)
print("GROUP →", grouped)


GROUP → {'Red': [1], 'Blue': [1], 'Green': [1, 1, 1]}


### REDUCE-PHASE
- Reduce values per key with a function f

In [ ]:
def reduceByKey(f, data):
    key_values = groupByKey(data)
    return list(map(lambda key: (key, foldl(f, key_values[key])), key_values))

reduced = reduceByKey(lambda x, y: x + y, mapped)
print("REDUCE →", reduced)

Folding 1 with [1, 1, 1, 1, 1]
Partial result is 2
Folding 2 with [1, 1, 1, 1]
Partial result is 3
Folding 3 with [1, 1, 1]
Partial result is 4
Folding 4 with [1, 1]
Partial result is 5
Folding 5 with [1]
Partial result is 6
REDUCE → [('Red', 1), ('Blue', 1), ('Green', 6), ('Yellow', 1), ('Black', 1), ('White', 1), ('Brown', 1), ('Orange', 1), ('Purple', 1), ('Pink', 1), ('Gray', 1)]


...now with tree-like `fold`

In [ ]:
def reduceByKey(f, data):
    key_values = groupByKey(data)
    return list(map(lambda key: (key, fold_tree(f, key_values[key])), key_values))

reduced = reduceByKey(lambda x, y: x + y, mapped)
print("REDUCE →", reduced)

Folding 1 with 1 → 2
Folding 1 with 1 → 2
Folding 1 with 1 → 2
Folding 2 with 2 → 4
Folding 4 with 2 → 6
REDUCE → [('Red', 1), ('Blue', 1), ('Green', 6), ('Yellow', 1), ('Black', 1), ('White', 1), ('Brown', 1), ('Orange', 1), ('Purple', 1), ('Pink', 1), ('Gray', 1)]


## Parallel Map Reduce

- Map Reduce in DFS was introduced for scalable parallel processing, i.e on big data with very large compute clusters.
- The most poular implementation is: e.g. Apache Hadoop (and Apache Spark).


- Hadoop MapReduce implementation will take care of the low-level functionality:
    - Task Scheduling & Load Balancing
    - Fault Tolerance & Recovery
    - Network & Disk I/O Management
    - Data Transfer Optimization and disk transfer optimisation
    - Cluster Resource Management
    - Transparency for Developers
    - etc..
- The model is designed to move the processing to where the data resides.

- All complexity of distributed execution, retries, I/O, and networking is abstracted away.

### Developers only implement map() and reduce() functions.

$\operatorname{map}(k, v) \rightarrow \; \left< k', v' \right>*$  

- and a reduce function:

$\operatorname{reduce}(k', \left< k', v'\right> *) \rightarrow \; \left< k', v''
\right> *$

- The $*$ refers to a *collection* of values.

- These collections are *not* ordered.

### Parallelizing Map Reduce

we will use multithreading, which will allow us to perform map-reduce computations that exploit parallel processing using *multiple* cores on a *single* computer.

- This approach (parallel reduce): splits the input into chunks that will be distributed to workers to be processes in parallel

```
INPUT DATA (16 Elemente)
        │
        ▼
+-------------------------------+
|  Split into 4 partitions      |
+-------------------------------+
   │         │        │       │
   ▼         ▼        ▼       ▼
 P0         P1       P2      P3
(MAP+RED) (MAP+RED) (MAP+RED)(MAP+RED)
   │         │        │       │
   └─────────┴────────┴───────┘
              │
              ▼
     Shuffle + Merge
              │
              ▼
       Final Global Reduce
              │
              ▼
    RESULT: [('Red',1),('Blue',1),('Green',6), ...]


(KI-generated flow-chart)
```



In [ ]:
from threading import Thread
from functools import reduce

colorlist = [
    'Red', 'Blue', 'Green', 'Green', 'Green', 'Green', 'Green', 'Green',
    'Yellow', 'Black', 'White', 'Brown', 'Orange', 'Purple', 'Pink', 'Gray'
]


1. Step: we split the input data into chunks

In [ ]:
def split_data(data, num_splits):
    n = len(data)
    k, m = divmod(n, num_splits)  # k = base size, m = remainder
    partitions = []
    start = 0
    for i in range(num_splits):
        end = start + k + (1 if i < m else 0)  # distribute remainder evenly
        partitions.append(data[start:end])
        start = end
    return partitions


In [ ]:
partitions = split_data(colorlist, 3)
print("PARTITIONS →", partitions)

PARTITIONS → [['Red', 'Blue', 'Green', 'Green', 'Green', 'Green'], ['Green', 'Green', 'Yellow', 'Black', 'White'], ['Brown', 'Orange', 'Purple', 'Pink', 'Gray']]


In [ ]:
# -----------------------------
# Parallel reduce across partitions
# -----------------------------
def parallel_reduce(f, partitions):
    n = len(partitions)
    results = [None] * n
    threads = [None] * n

    def job(i):
        mapped = map_step(partitions[i])              # map local
        reduced = reduceByKey(f, mapped)        # reduce local
        results[i] = reduced

    # Run each partition in a thread
    for i in range(n):
        threads[i] = Thread(target=lambda: job(i))
        threads[i].start()

    for i in range(n):
        threads[i].join()

    # Merge results from all partitions
    merged = []
    for r in results:
        merged.extend(r)

    # Final group + reduce
    final = reduceByKey(f, merged)
    return final

In [ ]:
result = parallel_reduce(lambda x, y: x + y, partitions)
print("FINAL RESULT →", result)

FoldingFolding 1 with [1]
Partial result is 2
 1 with [1, 1, 1]
Folding 1 with [1, 1]
Folding 1 with [1]
Partial result is 2
Partial result is 3
Partial result is 4
Folding 2 with [4]
Partial result is 6
FINAL RESULT → [('Red', 1), ('Blue', 1), ('Green', 6), ('Yellow', 1), ('Black', 1), ('White', 1), ('Brown', 1), ('Orange', 1), ('Purple', 1), ('Pink', 1), ('Gray', 1)]


# Next Step: parallel Map-Reduce in Hadoop

```
Input Data
   │
   ▼
Split into Blocks (Splits)
   │
   ▼
+------------+   +------------+
|  Mapper 1  |   |  Mapper 2  |
| (key,1)... |   | (key,1)... |
+------------+   +------------+
   │                  │
   └─────── Shuffle ──┘
           │
           ▼
   Partition by Key
   e.g.  Reducer 1 ← all "Green"
         Reducer 2 ← all "Blue"
         Reducer 3 ← all "Red"
           │
           ▼
       Reduce Step
           │
           ▼
     Final Results

     (KI-generated flow-chart)
```
